In [10]:
import numpy as np
from gc_ope.evaluate.utils.get_kde_estimator import get_kde_estimator_for_eval_res
from gc_ope.utils.train_log_process import extract_line_data, process_file

In [2]:
line_str = "[31m find max: point [231.0976473   -0.88249051  -5.30721331] with score 8.299604903511029e-08[0m from [3.48702921e-051 8.29960490e-008 8.64345998e-026 3.86571334e-024 7.09863806e-110 6.66722008e-046 4.12628213e-036 2.06915918e-185 9.14369746e-115 1.16077032e-038], score_threshold: 1.3227840819633532"
idx = line_str.find("find max")
line_str = line_str[idx:]
line_str

'find max: point [231.0976473   -0.88249051  -5.30721331] with score 8.299604903511029e-08\x1b from [3.48702921e-051 8.29960490e-008 8.64345998e-026 3.86571334e-024 7.09863806e-110 6.66722008e-046 4.12628213e-036 2.06915918e-185 9.14369746e-115 1.16077032e-038], score_threshold: 1.3227840819633532'

In [ ]:
extract_line_data(line_str)

In [3]:
process_log_file = "/home/gxd/pythonprojects/gc_ope/sac_omega_seed2.txt"

In [4]:
all_timestamps, all_dgs, all_scores = process_file(process_log_file, sample_goal_log_begin_strs=["find min", "find max"])

In [5]:
len(all_timestamps), len(all_dgs), len(all_scores)

(1503, 1503, 1503)

In [6]:
for a, b, c in zip(all_timestamps, all_dgs, all_scores):
    print(a, b, c)

10000 [222.34985478, -4.62215802, -2.44595661] 8.974750889207196e-09
10321 [217.14633958, -0.67039994, 6.36242083] 3.1527896578549104e-14
10321 [223.78627564, 1.16215872, -14.73636009] 0.0499502135296841
10321 [205.80091818, 1.36562389, -3.02170893] 2.511636201443475e-05
10321 [216.22590244, 2.89734809, -1.00293296] 1.146375395324114e-11
11295 [207.89893848, 2.248945, -8.74943392] 2.2196481909941974e-09
11295 [226.01594862, 0.25014218, -8.93183733] 9.245264279195307e-10
11295 [242.73694872, -5.3175751, -3.79375301] 0.003016827058736656
11295 [177.75264405, 1.39275264, -6.41464689] 6.475112121989948e-17
12645 [236.1940647, -5.37263893, -20.24407504] 1.3186049870455827e-08
12645 [176.17245509, -0.28667521, -4.12350558] 2.3072500436934134e-15
12645 [240.9574868, 0.40266734, -26.45836567] 5.0108404455202866e-11
12645 [248.67644542, 2.06881544, -27.18967535] 1.7444992183507485e-18
14161 [228.38493752, -8.23495776, -11.21129194] 9.665212548263462e-08
14161 [203.707897, -0.15213005, -13.55204

In [18]:
checkpoints_root_dir = "/home/gxd/pythonprojects/gc_ope/checkpoints/flycraft/easy/sac/seed_1"

timestamp_interval = 10000
cur_timestamp = 0

for tmp_timestamp, tmp_dg, tmp_score in zip(all_timestamps, all_dgs, all_scores):
    tmp = int(np.floor(int(tmp_timestamp) / timestamp_interval) * timestamp_interval)
    if tmp != cur_timestamp:
        cur_timestamp = tmp
        estimator = get_kde_estimator_for_eval_res(
            res_dir=f"{checkpoints_root_dir}/rl_model_{cur_timestamp}_steps_eval_res_on_fixed.csv",
            goal_keys_in_csv=["v", "mu", "chi"],
        )
        estimator.fit_evaluator()
    
    scaled_dgs, estimated_score = estimator.evaluate(
        desired_goals=[tmp_dg],
        scale=True,
        return_density=True,
    )

    print(tmp_timestamp, cur_timestamp, tmp_dg, tmp_score, estimated_score[0])

10000 10000 [222.34985478, -4.62215802, -2.44595661] 8.974750889207196e-09 0.0006029113814916127
10321 10000 [217.14633958, -0.67039994, 6.36242083] 3.1527896578549104e-14 0.0006832248760878598
10321 10000 [223.78627564, 1.16215872, -14.73636009] 0.0499502135296841 4.7221404577402094e-11
10321 10000 [205.80091818, 1.36562389, -3.02170893] 2.511636201443475e-05 0.010256306731405403
10321 10000 [216.22590244, 2.89734809, -1.00293296] 1.146375395324114e-11 0.008011925068387047
11295 10000 [207.89893848, 2.248945, -8.74943392] 2.2196481909941974e-09 7.395120811029584e-05
11295 10000 [226.01594862, 0.25014218, -8.93183733] 9.245264279195307e-10 1.7028020344860315e-06
11295 10000 [242.73694872, -5.3175751, -3.79375301] 0.003016827058736656 3.119405853554464e-09
11295 10000 [177.75264405, 1.39275264, -6.41464689] 6.475112121989948e-17 1.6471483364369859e-06
12645 10000 [236.1940647, -5.37263893, -20.24407504] 1.3186049870455827e-08 1.880931824084192e-21
12645 10000 [176.17245509, -0.28667521,